The main workspace for calculating pseudoinverse $A^\dagger$ of nonsymmetric matrices 
   
To do list: 
Make sure NewtonSchulz has the correct $\alpha$ in $\alpha A^T$
Perform tests on varying rank with fixed dimension

In [1]:
workspace()

In [1]:
using Plots
using JLD
include("pseudoinverse.jl")
include("ridge_regression_LIBSVM_data.jl")  
# IMPORTANT: before compiling, open dataLoad.jl and set the default_path variable 
include("dataLoad.jl")                      # Code by Dominik Csiba see  http://www.dominikcsiba.com/
initDetails()
include("../src/Pseudoinverse.jl")
importall Pseudoinverse

INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/FixedSizeArrays.ji for module FixedSizeArrays.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/FixedPointNumbers.ji for module FixedPointNumbers.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/PlotUtils.ji for module PlotUtils.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/PlotThemes.ji for module PlotThemes.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/Showoff.ji for module Showoff.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/FileIO.ji for module FileIO.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/JLD.ji for module JLD.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/StatsBase.ji for module StatsBase.
HDF5-DIAG: Error detected in HDF5 (1.8.11) thread 139677088427072:
  #000: ../../../src/H5F.c line 1504 in H5Fcreate(): unable to create file
    major: File accessibilty
    minor: Unable to open fil

LoadError: LoadError: Error creating file /local/rgower/libsvmdata/details.jld
while loading In[1], in expression starting on line 6

In [12]:
#Basic parameters
tol = 10.0^(-2); restol = 10.0^(-4);
maxiter=1000;
max_time = 500;
printiters = true;    #Print iterations
exacterror =false;    #Calculate error using ||X - A^\dagger ||

In [13]:
# uniform symmetrized low rank matrix
n = 5000;
r =1000;
prob = uniform_sym_rank(n,r); max_time = 200;

In [20]:
#ridge regression matrix using LIBSVM data
dataset = "rcv1_train.binary"; # a9a, mushrooms, gisette_scale
#transformDataJLD(dataset)
prob = ridge_regression_LIBSVM_data(dataset);
sA = size(prob.A);
n = sA[1];

In [21]:
skip_error_calculation = 2;    # number of iterations where error is not calculated (to save time!)
options = MyOptions(n,n,0 ,"uni",tol,restol, maxiter,skip_error_calculation,max_time,printiters,exacterror, "none") 

pseudoinverse.MyOptions(20242,20242,0,"uni",0.01,0.0001,1000,2,200.0,true,false,"none")

In [ ]:
    OUTPUTS = [];
    options.max_time =2000;
    method_name = "SAXAS";#        # Options of methods"SATAX"; "NewtonSchultz" "SAXAS"
    options.sketch = "ada";
    options.M0type = "ATAnorm";    # select initial matrix, options: eye, ATAnorm
    output= pseudoinvert_matrix(prob, method_name, options);
    OUTPUTS = [OUTPUTS ; output];

    options.sketch = "uni";
    options.M0type = "ATAnorm";
    output= pseudoinvert_matrix(prob, method_name, options);
    OUTPUTS = [OUTPUTS ; output];

    method_name = "NewtonSchulz";#"SATAX"; "NewtonSchultz" "SAXAS"
    output= pseudoinvert_matrix(prob, method_name, options);
    OUTPUTS = [OUTPUTS ; output];

In [23]:
savename = replace(prob.name, r"[\/]", "-");
save("./data/$(savename).jld", "OUTPUTS", OUTPUTS)

In [24]:
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)
save("./data/$(savename).jld", "OUTPUTS", OUTPUTS)

rcv1_train.binary-ridge


In [ ]:
# Load saved data
savename = "gisette_scale-ridge"
test = load("./data/$(savename).jld")
OUTPUTS = test["OUTPUTS"];